### Problem on Inventory Management
Prepared for: ExxpertSCM Thoughtware Training Pvt Ltd.


Guided by:  Mr. Pattabhi Raman

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import norm
import math

### problem 1: Optimal Policy for Discrete Demand

In [ ]:
def calculate_optimal_stock_level(purchase_price, selling_price, refund, demand,
                                  understock_cost, overstock_cost, mean_demand, std_demand):
    critical_ratio = understock_cost / (understock_cost + overstock_cost)

    # Calculate the optimal stock level using the inverse CDF (percent point function) of the normal distribution
    z = stats.norm.ppf(critical_ratio)
    optimal_stock_level = round(mean_demand + z * std_demand)

    return optimal_stock_level
# Calculate the critical ratio
purchase_price = float(input('Enter the purchase price:'))
selling_price = float(input('Enter the selling price:'))
refund = float(input('Enter the refund amount:'))
demand = list(map(int, input("Enter demand values (separated by commas):").replace(' ', '').split(',')))

understock_cost = selling_price - purchase_price
overstock_cost = purchase_price - refund
mean_demand = np.mean(demand)
std_demand = np.std(demand)

optimal_stock_level = calculate_optimal_stock_level(purchase_price, selling_price, refund, demand,
                                                    understock_cost, overstock_cost, mean_demand, std_demand)
print("Optimal Stock Level:", optimal_stock_level)


Enter the purchase price:25
Enter the selling price:75
Enter the refund amount:10
Enter demand values (separated by commas):15,19,9,12,9,22,4,7,8,11,14,11,6,11,9,18,10,0,14,12,8,15,14,18,17,4,4,5,9,8,6,7,12,15,15,19,9,10,9,16,17,14,14,19,17,15,18,11,11,8,13,12
Optimal Stock Level: 15


### Problem: 2 EOQ in All-Unit Discount Structure

In [ ]:
import pandas as pd
import math

def calculate_eoq(D, S, h, quantity_ranges, price_ranges):
    eoq_values = []

    for q_range, price in zip(quantity_ranges, price_ranges):
        q1, q2 = q_range
        ci = price

        if q2 == float('inf'):
            q2 = D  # Set q2 to the annual demand if it's infinity
        eoq = math.sqrt((2 * D * S) / (h * ci))
        eoq_values.append((q1, q2, eoq))

    return eoq_values

# Taking inputs
D = int(input('Enter annual demand: '))
S = float(input('Enter ordering cost: '))
h = float(input('Enter carrying cost per unit per year: '))
number_of_slab = int(input("Number of slabs: "))

quantity_ranges = []
price_ranges = []

for i in range(number_of_slab):
    q1 = int(input('Enter the lower limit of qi: '))
    q2_input = input('Enter the higher limit of qi ("inf" for infinity): ')
    price = int(input('Enter the price: '))

    if q2_input.lower() == "inf":
        q2 = float('inf')
    else:
        q2 = float(q2_input)

    quantity_ranges.append((q1, q2))
    price_ranges.append(price)

# Calculate EOQ values
eoq_values = calculate_eoq(D, S, h, quantity_ranges, price_ranges)

# Choose optimal EOQ (Q*) based on criteria
optimal_eoq_values = []

for q_range, eoq in zip(quantity_ranges, eoq_values):
    q1, q2, eoq_value = eoq

    if eoq_value < q1:
        optimal_eoq = q1
    elif eoq_value >= q1 and eoq_value < q2:
        optimal_eoq = eoq_value
    else:
        optimal_eoq = q2

    optimal_eoq_values.append(optimal_eoq)

# Calculate total cost for each case using the formula TC = (D/optimalEOQ)*S + (optimalEOQ/2)*C*h + D*Ci
total_costs = []

for q_range, optimal_eoq, price in zip(quantity_ranges, optimal_eoq_values, price_ranges):
    q1, q2 = q_range
    ci = price

    total_cost = (D / optimal_eoq) * S + (optimal_eoq / 2) * ci * h + D * ci
    total_costs.append(total_cost)


# Create a DataFrame
data = {
    "Quantity Range": quantity_ranges,
    "Price": price_ranges,
    "Q* (EOQ based on criteria)": optimal_eoq_values,
    "Optimal EOQ": [eoq[2] for eoq in eoq_values],
    "Total Cost": total_costs
}

df = pd.DataFrame(data)

# Print the DataFrame
df


Enter annual demand: 1000
Enter ordering cost: 100
Enter carrying cost per unit per year: 0.20
Number of slabs: 3
Enter the lower limit of qi: 0
Enter the higher limit of qi ("inf" for infinity): 500
Enter the price: 100
Enter the lower limit of qi: 500
Enter the higher limit of qi ("inf" for infinity): 1000
Enter the price: 95
Enter the lower limit of qi: 1000
Enter the higher limit of qi ("inf" for infinity): inf
Enter the price: 90


,Quantity Range,Price,Q* (EOQ based on criteria),Optimal EOQ,Total Cost
0,"(0, 500.0)",100,100.0,100.000000,102000.0
1,"(500, 1000.0)",95,500.0,102.597835,99950.0
2,"(1000, inf)",90,1000.0,105.409255,99100.0


### Problem: 3 EOQ in Marginal-Unit Discount Structure

In [ ]:
import math
import pandas as pd
def calculate_Vi(q_values, ci_values):
    Vi_values = [0]  # Initialize with V0 = 0

    for i in range(1, len(q_values)):
        delta_q = q_values[i] - q_values[i - 1]
        Vi = delta_q * ci_values[i - 1] + Vi_values[i - 1]
        Vi_values.append(Vi)

    return Vi_values

# Take user input for q_values
q_values = [float(q) for q in input("Enter q values (comma-separated): ").split(",")]

# Take user input for ci_values
ci_values = [float(ci) for ci in input("Enter ci values (comma-separated): ").split(",")]

# Calculate Vi values
Vi_values = calculate_Vi(q_values, ci_values)

# Print the calculated Vi values
for i, Vi in enumerate(Vi_values):
    print(f"V{i} = {Vi:.2f}")


# Given data
q_values = [0, 100, 200]
ci_values = [500.5, 490.5, 475.5]

# Calculate Vi values
Vi_values = calculate_Vi(q_values, ci_values)

def calculate_EOQ(D, S, Vi, qi, Ci, h):
    return math.sqrt((2 * D * (S + Vi - qi * Ci)) / (h * Ci))

def calculate_optimal_EOQ(q_values, ci_values, Vi_values, D, S, h):
    optimal_EOQ_values = []

    for i in range(len(q_values)):
        qi = q_values[i]
        Ci = ci_values[i]
        Vi = Vi_values[i] if i < len(Vi_values) else Vi_values[-1]  # Extend Vi_values if needed

        EOQ = calculate_EOQ(D, S, Vi, qi, Ci, h)

        if i == len(q_values) - 1:
            optimal_EOQ_values.append(EOQ)  # Last value
        else:
            next_qi = q_values[i + 1]

            if qi <= EOQ < next_qi:
                optimal_EOQ_values.append(EOQ)
            elif EOQ < qi:
                optimal_EOQ_values.append(qi)
            else:
                optimal_EOQ_values.append(next_qi)

    return optimal_EOQ_values

# Take user input for D, S, and h
D = float(input("Enter annual demand (D): "))
S = float(input("Enter setup cost (S): "))
h = float(input("Enter holding cost per unit per year (h): "))

# Given data
q_values = [0, 100, 200]
ci_values = [500.5, 490.5, 475.5]

# Calculate Vi values
Vi_values = calculate_Vi(q_values, ci_values)

# Calculate optimal EOQ values
optimal_EOQ_values = calculate_optimal_EOQ(q_values, ci_values, Vi_values, D, S, h)
optimal_EOQ_values = [round(EOQ) for EOQ in optimal_EOQ_values]

# Initialize lists to store data for DataFrame
data = {
    "D": [D] * len(q_values),
    "S": [S] * len(q_values),
    "Ci": ci_values,
    "Vi": Vi_values,
    "Qi": q_values,
    "Optimal Qi": optimal_EOQ_values
}

# Calculate costs and add them to the data dictionary
ordering_costs = [(D / EOQ) * S for EOQ in optimal_EOQ_values]
inventory_carrying_costs = [(Vi + (EOQ - qi) * Ci) * (h / 2) for EOQ, qi, Ci, Vi in zip(optimal_EOQ_values, q_values, ci_values, Vi_values)]
material_costs = [(D * (Vi + (EOQ - qi) * Ci)) / EOQ for EOQ, qi, Ci, Vi in zip(optimal_EOQ_values, q_values, ci_values, Vi_values)]
total_costs = [ordering + carrying + material for ordering, carrying, material in zip(ordering_costs, inventory_carrying_costs, material_costs)]

data["Ordering Cost"] = ordering_costs
data["Inventory Carrying Cost"] = inventory_carrying_costs
data["Material Cost"] = material_costs
data["Total Cost"] = total_costs

# Create DataFrame
df = pd.DataFrame(data)

# Print the DataFrame
df


Enter q values (comma-separated): 0,100,200
Enter ci values (comma-separated): 500.5,490.5,475.5
V0 = 0.00
V1 = 50050.00
V2 = 99100.00
Enter annual demand (D): 1200
Enter setup cost (S): 25
Enter holding cost per unit per year (h): 0.2


,D,S,Ci,Vi,Qi,Optimal Qi,Ordering Cost,Inventory Carrying Cost,Material Cost,Total Cost
0,1200.0,25.0,500.5,0.0,0,24,1250.000000,1201.20,600600.000000,603051.200000
1,1200.0,25.0,490.5,50050.0,100,158,189.873418,7849.90,596194.936709,604234.710127
2,1200.0,25.0,475.5,99100.0,200,319,94.043887,15568.45,585647.021944,601309.515831


### Problem: 4 EOQ models with backorders

In [ ]:
import math

def calculate_QB(D, S, H, back_ordercost):
    Q = round(math.sqrt((2 * D * S) / H) * math.sqrt((H + back_ordercost) / back_ordercost))
    B = round(Q * (H / (H + back_ordercost)))
    return Q, B

def calculate_EOQ(D, S, H):
    Q_eoq = round(math.sqrt((2 * D * S) / H))
    return Q_eoq

def calculate_M_TC(Q, B, H):
    M = Q - B
    TC = (D / Q) * S + (M / 2) * H * (M / Q) + (B / 2) * back_ordercost * (B / Q)
    return M, TC


# Input values
D = float(input("Enter annual demand (D): "))
S = float(input("Enter setup cost (S): "))
H = float(input("Enter annual inventory carrying cost (H): "))
back_ordercost = float(input("Enter shortage cost per unit of time short (pi): "))


# Calculate Q and B using the original method

Q, B = calculate_QB(D, S, H, back_ordercost)
M, TC = calculate_M_TC(Q, B, H)

# Calculate EOQ and corresponding total cost
Q_eoq = calculate_EOQ(D, S, H)
TC_eoq = (D / Q_eoq) * S + (Q_eoq / 2) * H

# Calculate Q and B for backorder case
Q, B = calculate_QB(D, S, H, back_ordercost)
M, TC = calculate_M_TC(Q, B, H)

# Calculate the difference in total costs
cost_difference = TC - TC_eoq

print(f"Backorder scenario:")
print(f"Q value (Q): {Q}")
print(f"Backorder quantity (B): {B}")
print(f"Total cost (TC): {TC:.2f}")

print("\nEOQ scenario:")
print(f"EOQ value (Q): {Q_eoq}")
print(f"Total cost (TC): {TC_eoq}")

print(f"Difference in total costs (Original - Backorder): {cost_difference}")


Enter annual demand (D): 10000
Enter setup cost (S): 500
Enter annual inventory carrying cost (H): 10
Enter shortage cost per unit of time short (pi): 40
Backorder scenario:
Q value (Q): 1118
Backorder quantity (B): 224
Total cost (TC): 8944.28

EOQ scenario:
EOQ value (Q): 1000
Total cost (TC): 10000.0
Difference in total costs (Original - Backorder): -1055.7245080500888


### Problem: 5 Conditions of Uncertainty: Discrete distributions

In [ ]:
import math

def calculate_total_cost(D, S, h, C):
    Q = math.sqrt((2 * D * S) / (h * C))
    cycles = D / Q
    total_cost = (D / Q) * S + (Q / 2) * C * h
    return Q, cycles, total_cost

# Input values
D = float(input("Enter annual demand (D): "))
S = float(input("Enter setup cost (S): "))
h = float(input("Enter carrying cost (h): "))
C = float(input("Enter unit price (C): "))

Q, cycles, total_cost = calculate_total_cost(D, S, h, C)

print(f"Q: {Q}")
print(f"Number of cycles: {cycles}")
print(f"Total cost: {total_cost}")


Enter annual demand (D): 3600
Enter setup cost (S): 200
Enter carrying cost (h): 0.25
Enter unit price (C): 100
Q: 240.0
Number of cycles: 15.0
Total cost: 6000.0


In [ ]:
import numpy as np

# Given demand and probability data
demand_values = [100, 110, 120, 130, 140, 150, 160]
demand_probabilities = [0.01, 0.06, 0.24, 0.38, 0.24, 0.06, 0.01]

# Reorder point matrix values
reorder_points = [
    [0, 10, 20, 30, 40, 50, 60],
    [-10, 0, 10, 20, 30, 40, 50],
    [-20, -10, 0, 10, 20, 30, 40],
    [-30, -20, -10, 0, 10, 20, 30],
    [-40, -30, -20, -10, 0, 10, 20],
    [-50, -40, -30, -20, -10, 0, 10],
    [-60, -50, -40, -30, -20, -10, 0]
]

# Calculate unit of excess and shortages for each reorder point
unit_excess_and_shortage = np.zeros((len(reorder_points), len(reorder_points)))

for i in range(len(reorder_points)):
    for j in range(len(reorder_points)):
        unit_excess_and_shortage[j][i] = reorder_points[j][i] * demand_probabilities[j]

# Display demand and probability table
print("Demand and Probability Table:")
print(" Demand | Probability")
print("----------------------")
for demand, prob in zip(demand_values, demand_probabilities):
    print(f"   {demand}   |    {prob}")

# Display reorder point matrix
print("\nReorder Point Matrix:")
for row in reorder_points:
    print(row)

# Display unit of excess and shortages table
print("\nUnit of Excess and Shortages Table:")
print(" Reorder Points |", end="")
for demand in demand_values:
    print(f" {demand:3d} ", end="")
print("\n----------------|", end="")
for demand in demand_values:
    print("----", end="")
print()
for i in range(len(reorder_points)):
    print(f" {demand_values[i]:15d} |", end="")
    for j in range(len(reorder_points)):
        print(f" {unit_excess_and_shortage[i][j]:4.2f} ", end="")
    print()



Demand and Probability Table:
 Demand | Probability
----------------------
   100   |    0.01
   110   |    0.06
   120   |    0.24
   130   |    0.38
   140   |    0.24
   150   |    0.06
   160   |    0.01

Reorder Point Matrix:
[0, 10, 20, 30, 40, 50, 60]
[-10, 0, 10, 20, 30, 40, 50]
[-20, -10, 0, 10, 20, 30, 40]
[-30, -20, -10, 0, 10, 20, 30]
[-40, -30, -20, -10, 0, 10, 20]
[-50, -40, -30, -20, -10, 0, 10]
[-60, -50, -40, -30, -20, -10, 0]

Unit of Excess and Shortages Table:
 Reorder Points | 100  110  120  130  140  150  160 
----------------|----------------------------
             100 | 0.00  0.10  0.20  0.30  0.40  0.50  0.60 
             110 | -0.60  0.00  0.60  1.20  1.80  2.40  3.00 
             120 | -4.80  -2.40  0.00  2.40  4.80  7.20  9.60 
             130 | -11.40  -7.60  -3.80  0.00  3.80  7.60  11.40 
             140 | -9.60  -7.20  -4.80  -2.40  0.00  2.40  4.80 
             150 | -3.00  -2.40  -1.80  -1.20  -0.60  0.00  0.60 
             160 | -0.60  -0.50  

In [ ]:
# Calculate total excess (e) and total shortage (g) values
total_shortage_values = [abs(sum(unit_excess_and_shortage[j][i] for j in range(i + 1, len(reorder_points)))) for i in range(len(demand_values))]
total_excess_values = [abs(sum(unit_excess_and_shortage[j][i] for j in range(i)) + unit_excess_and_shortage[i][i]) for i in range(len(demand_values))]

# Calculate excess cost (E) and shortage cost per cycle (G)
H = h*C # Expected carrying cost
back_ordercost = float(input("Enter shortage cost per unit of time short (pi): "))

excess_cost_values = [e * H for e in total_excess_values]
shortage_cost_values = [g * back_ordercost for g in total_shortage_values]

total_annual_shortage_costs = [shortage_cost * cycles for shortage_cost in shortage_cost_values]

# Calculate total annual cost for each demand
total_annual_costs = [shortage_cost + excess_cost for shortage_cost, excess_cost in zip(total_annual_shortage_costs, excess_cost_values)]

# Create a DataFrame
data = {
    'Demand': demand_values,
    'Total Excess (e)': total_excess_values,
    'Excess Cost (E)': excess_cost_values,
    'Total Shortage (g)': total_shortage_values,
    'Shortage Cost per Cycle (G)': shortage_cost_values,
    'Total Annual Shortage Cost': total_annual_shortage_costs,
    'Total Annual Cost': total_annual_costs

}

df = pd.DataFrame(data)

# Display the DataFrame
df

Enter shortage cost per unit of time short (pi): 10


,Demand,Total Excess (e),Excess Cost (E),Total Shortage (g),Shortage Cost per Cycle (G),Total Annual Shortage Cost,Total Annual Cost
0,100,0.0,0.0,30.0,300.0,4500.0,4500.0
1,110,0.1,2.5,20.1,201.0,3015.0,3017.5
2,120,0.8,20.0,10.8,108.0,1620.0,1640.0
3,130,3.9,97.5,3.9,39.0,585.0,682.5
4,140,10.8,270.0,0.8,8.0,120.0,390.0
5,150,20.1,502.5,0.1,1.0,15.0,517.5
6,160,30.0,750.0,0.0,0.0,0.0,750.0


In [ ]:
# Find the row with the lowest total annual cost
lowest_cost_row = df.loc[df['Total Annual Cost'].idxmin()]

# Extract the total shortage (g) value from the lowest cost row
lowest_g = lowest_cost_row['Total Shortage (g)']
lowest_e = lowest_cost_row['Total Excess (e)']
# Calculate Q* using the provided formula
Q_star = round(math.sqrt((2 * D * (S + lowest_g * back_ordercost)) / H))

total_annual_cost_lowest = round((D / Q_star) * S + (Q_star / 2) * H + lowest_e * H + (D/Q_star)*lowest_g*back_ordercost)
# Display the results
print(f"Lowest Total Annual Cost Row:")
print(lowest_cost_row)
print(f"Q*: {Q_star}")
print(f"Total Annual Cost (Lowest): {total_annual_cost_lowest}")


Lowest Total Annual Cost Row:
Demand                         140.0
Total Excess (e)                10.8
Excess Cost (E)                270.0
Total Shortage (g)               0.8
Shortage Cost per Cycle (G)      8.0
Total Annual Shortage Cost     120.0
Total Annual Cost              390.0
Name: 4, dtype: float64
Q*: 245
Total Annual Cost (Lowest): 6389


### Problem: 6 Order service Level and Unit Service Level

In [ ]:
# Input the desired range
total_period = int(input("Enter the range(total_period)value: "))

data = {
    'period': list(range(1, total_period + 1)),
    'demand': [None] * total_period ,  # Placeholder for demand values
    'quantity_sold': [None] * total_period,
    'quantity_left': [None] * total_period,
    'lost_sales': [None] * total_period,
    'stock_out_during_period': [None] * total_period
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Input demand values using a loop
for period in df['period']:
    df.loc[df['period'] == period, 'demand'] = float(input(f"Enter demand for period {period}: "))

Enter the range(total_period)value: 50
Enter demand for period 1: 37
Enter demand for period 2: 37
Enter demand for period 3: 75
Enter demand for period 4: 73
Enter demand for period 5: 53
Enter demand for period 6: 47
Enter demand for period 7: 44
Enter demand for period 8: 41
Enter demand for period 9: 37
Enter demand for period 10: 35
Enter demand for period 11: 25
Enter demand for period 12: 15
Enter demand for period 13: 12
Enter demand for period 14: 43
Enter demand for period 15: 72
Enter demand for period 16: 29
Enter demand for period 17: 40
Enter demand for period 18: 43
Enter demand for period 19: 50
Enter demand for period 20: 41
Enter demand for period 21: 53
Enter demand for period 22: 47
Enter demand for period 23: 57
Enter demand for period 24: 15
Enter demand for period 25: 45
Enter demand for period 26: 66
Enter demand for period 27: 33
Enter demand for period 28: 58
Enter demand for period 29: 76
Enter demand for period 30: 31
Enter demand for period 31: 35
Enter dem

In [ ]:
# Input the keeping_unit value
keeping_unit = float(input("Enter the keeping unit value: "))

# Calculate additional columns
df['quantity_sold'] = df.apply(lambda row: min(row['demand'], keeping_unit), axis=1)
df['quantity_left'] = keeping_unit - df['quantity_sold']
df['lost_sales'] = df.apply(lambda row: max(row['demand'] - row['quantity_sold'], 0), axis=1)
df['stock_out_during_period'] = df['quantity_left'].apply(lambda x: 1 if x == 0 else 0)

# Print the updated DataFrame
df

Enter the keeping unit value: 45


,period,demand,quantity_sold,quantity_left,lost_sales,stock_out_during_period
0,1,37.0,37.0,8.0,0.0,0
1,2,37.0,37.0,8.0,0.0,0
2,3,75.0,45.0,0.0,30.0,1
3,4,73.0,45.0,0.0,28.0,1
4,5,53.0,45.0,0.0,8.0,1
5,6,47.0,45.0,0.0,2.0,1
6,7,44.0,44.0,1.0,0.0,0
7,8,41.0,41.0,4.0,0.0,0
8,9,37.0,37.0,8.0,0.0,0
9,10,35.0,35.0,10.0,0.0,0


In [ ]:
def calculate_metrics(df):
    sum_demand = df['demand'].sum()
    sum_lost_sales = df['lost_sales'].sum()
    sum_stock_out = df['stock_out_during_period'].sum()

    print(f"Sum of Demand: {sum_demand}")
    print(f"Sum of Lost Sales: {sum_lost_sales}")
    print(f"Sum of Stock Out During Period: {sum_stock_out}")


    total_stock_out_situvation = total_period - sum_stock_out

    order_service_level = total_stock_out_situvation / total_period
    OSOR = 1 - order_service_level

    USL = 1 - (sum_lost_sales / sum_demand)

    return sum_demand, sum_lost_sales, sum_stock_out, total_stock_out_situvation, order_service_level, OSOR, USL

In [ ]:
# Call the function and provide your DataFrame as an argument
metrics = calculate_metrics(df)

Sum of Demand: 2012.0
Sum of Lost Sales: 291.0
Sum of Stock Out During Period: 20


In [ ]:
# Unpack the returned metrics
sum_demand, sum_lost_sales, sum_stock_out, total_stock_out_situvation, order_service_level, OSOR, USL = metrics

In [ ]:
# Print the calculated metrics
print(f"total_stock_out_situvation: {total_stock_out_situvation}")
print(f"order_service_level: {order_service_level}")
print(f"OSOR: {OSOR}")
print(f"USL: {USL}")

total_stock_out_situvation: 30
order_service_level: 0.6
OSOR: 0.4
USL: 0.8553677932405567


### Problem: 7 Determining Reorder Point when Demand is uncertain

In [ ]:
def metrics(d,Sd,C,LT,S,P,h):
    EOQ = round(np.sqrt((2 * d * S) / (h / 12 * C)))
    z = round(norm.ppf(P),2)
    safety_stock = round(z * Sd * np.sqrt(LT))
    d_prime = round(d * LT)
    sd_prime = round(Sd * np.sqrt(LT))
    ROP =round(d_prime + safety_stock)
    return EOQ, z, safety_stock, d_prime, sd_prime, ROP
    ## inputing values
d = float(input('enter the monthly demnad: '))
Sd = float(input('enter the standard deviation of demand: '))
C = float(input('enter the unit cost of the item: '))
LT = float(input('enter the lead time: '))
S = float(input('enter the acquisition cost for order processing: '))
P = float(input('in-stock probability: '))
h = float(input('inventory carrying cost: '))

enter the monthly demnad: 11000
enter the standard deviation of demand: 3500
enter the unit cost of the item: 0.15
enter the lead time: 1.5
enter the acquisition cost for order processing: 20
in-stock probability: 0.75
inventory carrying cost: 0.25


In [ ]:
metrics = metrics(d,Sd,C,LT,S,P,h)
EOQ, z, safety_stock, d_prime, sd_prime, ROP = metrics

In [ ]:
print(f"EOQ: {EOQ}")
print(f"z: {z}")
print(f"Safety Stock: {safety_stock}")
print(f"Reorder Point (ROP): {ROP}")

EOQ: 11866
z: 0.67
Safety Stock: 2872
Reorder Point (ROP): 19372


### Problem: 8 What If Scenario in Inventory Decisions

In [ ]:
d = float(input('Enter the weekly demand: '))
Sd = float(input('Enter the std of demand: '))
LT = float(input('Enter the Lead time in weeks: '))
ROP = float(input('Enter the reorder point: '))
order_quantity = float(input('Enter the order quantity for the given ROP: '))

Enter the weekly demand: 3000
Enter the std of demand: 600
Enter the Lead time in weeks: 2
Enter the reorder point: 7000
Enter the order quantity for the given ROP: 9000


In [ ]:
def calculate(d,Sd,LT,ROP,order_quantity):
    # Calculate d' and sd'
    d_prime = d * LT
    sd_prime = round(Sd * np.sqrt(LT))

    # Calculate safety stock from ROP equation
    safety_stock = ROP - d_prime

    # Calculate z from safety stock and sd'
    z = safety_stock / sd_prime

    # Calculate E(z)
    pdf = norm.pdf(z)
    cdf = norm.cdf(z)
    unl = pdf - z * (1 - cdf)

    # Calculate USL
    usl = 1 - (unl * sd_prime / order_quantity)
    return d_prime, sd_prime, safety_stock, z, unl, usl

In [ ]:
metrics = calculate(d,Sd,LT,ROP,order_quantity)
d_prime, sd_prime, safety_stock, z, unl, usl = metrics

In [ ]:
# Display results
print(f"Safety Stock: {safety_stock:.2f} units")
print(f"z: {z:.2f}")
print(f"Unit Normal Loss (UNL): {unl:.4f}")
print(f"USL: {usl:.4f}")

Safety Stock: 1000.00 units
z: 1.18
Unit Normal Loss (UNL): 0.0587
USL: 0.9945


Problem 9

In [ ]:
## import pandas as pd
import numpy as np
from scipy.stats import norm
import math

def calculate_inventory_parameters(D, S, H, d_prime, Sd_prime, OSL):
    Q = math.sqrt((2 * D * S) / H)
    z = round(norm.ppf(OSL), 2)
    ROP = d_prime + (z * Sd_prime)
    SS = z * Sd_prime
    cdf = norm.cdf(z)
    pdf = norm.pdf(z)
    E_z = round(pdf - z * (1 - cdf), 2)
    USL = round(1 - ((E_z * Sd_prime) / S), 2)
    USOR = round(1 - USL, 2)
    OSOR = round(1 - OSL, 3)
    pi = (H * Q) / (OSOR * D)

    result = {
        'Quantity (Q)': Q,
        'Z-score': z,
        'Reorder Point (ROP)': ROP,
        'Safety Stock (SS)': SS,
        'Unit Normal Loss Value (E_z)': E_z,
        'Unit Service Level (USL)': USL,
        'Unit Stock Out Risk (USOR)': USOR,
        'Order Stock Out Risk (OSOR)': OSOR,
        'Backorder Penalty Cost (pi)': pi,
    }

    return result

# Taking user inputs
D = float(input('Enter the demand: '))
S = float(input('Enter the set up cost: '))
H = float(input('Enter the carrying cost: '))
d_prime = float(input("Enter the d': "))
Sd_prime = float(input("Enter the Sd': "))
OSL = float(input('Enter the order service level: '))

# Calculate and print the results
results = calculate_inventory_parameters(D, S, H, d_prime, Sd_prime, OSL)

# Calculate and print the results
results = calculate_inventory_parameters(D, S, H, d_prime, Sd_prime, OSL)
for key, value in results.items():
    print(key, ":", value)


Enter the demand: 15000
Enter the set up cost: 1000
Enter the carrying cost: 30
Enter the d': 300
Enter the Sd': 50
Enter the order service level: 0.70
Quantity (Q) : 1000.0
Z-score : 0.52
Reorder Point (ROP) : 326.0
Safety Stock (SS) : 26.0
Unit Normal Loss Value (E_z) : 0.19
Unit Service Level (USL) : 0.99
Unit Stock Out Risk (USOR) : 0.01
Order Stock Out Risk (OSOR) : 0.3
Backorder Penalty Cost (pi) : 6.666666666666667


Problem: 10

In [ ]:
import scipy.stats as stats
import scipy.optimize as optimize
import math

# input parameters
D = float(input("Enter the demand (D): "))
H = float(input("Enter the carrying cost (H): "))
S = float(input("Enter the setup cost (S): "))
d_prime = float(input("Enter d_prime: "))
sd_prime = float(input("Enter sd_prime: "))
USL = float(input("Enter the USL: "))
# Initial values
Q = math.sqrt((2 * D * S) / H)
USOR = 1 - USL
E_z = (Q * USOR) / sd_prime
iteration = 0

# Convergence criterion
convergence_threshold = 0.001

while True:
    # Step 3: Solve for z
    equation_to_solve = lambda z: stats.norm.pdf(z) - z * (1 - stats.norm.cdf(z)) - E_z
    result = optimize.root(equation_to_solve, 0.0)
    z_value = result.x[0]

    # Step 4: Calculate new Q and back order cost
    OSL = stats.norm.cdf(z_value)
    OSOR = 1 - OSL
    back_ordercost = (H * Q) / (OSOR * D)
    Q_star = math.sqrt((2 * D * (S + back_ordercost * E_z * sd_prime)) / H)

    # Check for convergence
    if abs(Q - Q_star) < convergence_threshold or iteration >= 100:
        break

    Q = Q_star
    E_z = (Q * USOR) / sd_prime
    iteration += 1


print("Final Q_star:", Q_star)
print("Final E_z:", E_z)
print("Final Back Order Cost:", back_ordercost)


Enter the demand (D): 15000
Enter the carrying cost (H): 30
Enter the setup cost (S): 1000
Enter d_prime: 200
Enter sd_prime: 50
Enter the USL: 0.95
Final Q_star: 1065.7522664861108
Final E_z: 1.0657517679011133
Final Back Order Cost: 2.548959290732047
